# **Noisy Double DQN Reinforcement Learning Agent for Blackjack**
In this notebook, we evaluate the noisey trained model using the NoisyDoubleDQN class within our training notebook. In our training, we used 500,000 epochs to train the agent using off-policy learning with a replaymemory.

## Imports and Installs

In [ ]:
%%capture
# capture line is to hide the output
# Install required packages
!pip install gymnasium torch numpy matplotlib

In [ ]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from collections import deque
import random
import math
import random
from collections import namedtuple, deque

In [ ]:
import gdown

## **Noisy Double DQN Agent with Replay Memory Implementation**
The Agent and the ReplayMemory class are instantiated here, and then the model weights are loaded. For additional improvement, it might be beneficial to save the agent and replaymemory as a class within a .py file for import.

In [ ]:
# Define a named tuple to store experiences
Experience = namedtuple('Experience', ('state', 'action', 'reward', 'next_state', 'done'))

class ReplayMemory:
    def __init__(self, capacity=10000):
        self.memory = deque(maxlen=capacity)
        self.capacity = capacity

    def push(self, state, action, reward, next_state, done):
        """Save an experience to memory"""
        experience = Experience(state, action, reward, next_state, done)
        self.memory.append(experience)

    def sample(self, batch_size):
        """Randomly sample a batch of experiences from memory"""
        if batch_size > len(self.memory):
            batch_size = len(self.memory)
        experiences = random.sample(self.memory, batch_size)

        # Convert to separate arrays
        states = torch.FloatTensor([exp.state for exp in experiences])
        actions = torch.LongTensor([exp.action for exp in experiences])
        rewards = torch.FloatTensor([exp.reward for exp in experiences])
        next_states = torch.FloatTensor([exp.next_state for exp in experiences])
        dones = torch.FloatTensor([exp.done for exp in experiences])

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.memory)

# Initialize replay memory
memory = ReplayMemory()

# Test the replay memory with a sample experience
sample_experience = (
    np.array([11, 9, 0]),  # state
    1,                     # action (hit)
    0,                     # reward
    np.array([16, 9, 0]),  # next_state
    False                  # not done
)

memory.push(*sample_experience)
print(f"Memory size after pushing one experience: {len(memory)}")

# Test sampling (this will only return the one experience we pushed)
states, actions, rewards, next_states, dones = memory.sample(1)
print("\nSampled experience:")
print(f"State: {states}")
print(f"Action: {actions}")
print(f"Reward: {rewards}")
print(f"Next State: {next_states}")
print(f"Done: {dones}")

Memory size after pushing one experience: 1

Sampled experience:
State: tensor([[11.,  9.,  0.]])
Action: tensor([1])
Reward: tensor([0.])
Next State: tensor([[16.,  9.,  0.]])
Done: tensor([0.])


In [ ]:
class NoisyLinear(nn.Module):
    """Noisy linear module for NoisyNet.

    Attributes:
        in_features (int): input size of linear module
        out_features (int): output size of linear module
        std_init (float): initial std value
        weight_mu (nn.Parameter): mean value weight parameter
        weight_sigma (nn.Parameter): std value weight parameter
        bias_mu (nn.Parameter): mean value bias parameter
        bias_sigma (nn.Parameter): std value bias parameter

    """

    def __init__(self, in_features: int, out_features: int, std_init: float = 0.5):
        """Initialization."""
        super(NoisyLinear, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init

        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weight_sigma = nn.Parameter(
            torch.Tensor(out_features, in_features)
        )
        self.register_buffer(
            "weight_epsilon", torch.Tensor(out_features, in_features)
        )

        self.bias_mu = nn.Parameter(torch.Tensor(out_features))
        self.bias_sigma = nn.Parameter(torch.Tensor(out_features))
        self.register_buffer("bias_epsilon", torch.Tensor(out_features))

        self.reset_parameters()
        self.reset_noise()

    def reset_parameters(self):
        """Reset trainable network parameters (factorized gaussian noise)."""
        mu_range = 1 / math.sqrt(self.in_features)
        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(
            self.std_init / math.sqrt(self.in_features)
        )
        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(
            self.std_init / math.sqrt(self.out_features)
        )

    def reset_noise(self):
        """Make new noise."""
        epsilon_in = self.scale_noise(self.in_features)
        epsilon_out = self.scale_noise(self.out_features)

        # outer product
        self.weight_epsilon.copy_(epsilon_out.ger(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward method implementation.

        We don't use separate statements on train / eval mode.
        It doesn't show remarkable difference of performance.
        """
        return F.linear(
            x,
            self.weight_mu + self.weight_sigma * self.weight_epsilon,
            self.bias_mu + self.bias_sigma * self.bias_epsilon,
        )

    @staticmethod
    def scale_noise(size: int) -> torch.Tensor:
        """Set scale to make noise (factorized gaussian noise)."""
        x = torch.randn(size)

        return x.sign().mul(x.abs().sqrt())

## Noisy DDQN Architecture

In [ ]:
import torch.optim as optim

class NoisyDoubleDQNAgent:
    def __init__(self, input_dim=3, learning_rate=5e-4, gamma=0.99, epsilon=1.0):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.fitness = [] # used to store a series of 'avg reward' during evaluation

        self.policy_noisy_linear1 = NoisyLinear(128, 64)
        self.policy_noisy_linear2 = NoisyLinear(64, 2)
        self.target_noisy_linear1 = NoisyLinear(128, 64)
        self.target_noisy_linear2 = NoisyLinear(64, 2)

        # Larger network
        self.policy_net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            self.policy_noisy_linear1, # the noisy layers w/o ReLU in-between
            self.policy_noisy_linear2,
        ).to(self.device)

        self.target_net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            self.target_noisy_linear1,
            self.target_noisy_linear2
        ).to(self.device)

        self.target_net.load_state_dict(self.policy_net.state_dict())

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.memory = ReplayMemory(capacity=20000)

        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.99997
        self.batch_size = 128
        self.target_update = 10
        self.episode_count = 0


    def reset_noise(self):
        """Reset all noisy layers."""
        self.policy_noisy_linear1.reset_noise()
        self.policy_noisy_linear2.reset_noise()
        self.target_noisy_linear1.reset_noise()
        self.target_noisy_linear2.reset_noise()

    def select_action(self, state):
        """Select action using epsilon-greedy policy"""
        if random.random() < self.epsilon:
            return random.randint(0, 1)

        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.policy_net(state)
            return q_values.argmax().item()

    def update_epsilon(self):
        """Decay epsilon value"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def store_transition(self, state, action, reward, next_state, done):
        """Store transition in replay memory"""
        self.memory.push(state, action, reward, next_state, done)

    def train_step(self):
        """Perform one training step"""
        if len(self.memory) < self.batch_size:
            return

        states, actions, rewards, next_states, dones = self.memory.sample(self.batch_size)
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)

        # Double DQN implementation
        with torch.no_grad():
            next_actions = self.policy_net(next_states).argmax(1)
            next_q_values = self.target_net(next_states).gather(1, next_actions.unsqueeze(1)).squeeze(1)
            target_q_values = rewards + (1 - dones.float()) * self.gamma * next_q_values

        current_q_values = self.policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)

        # Huber loss for better stability
        loss = nn.SmoothL1Loss()(current_q_values, target_q_values)

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()

        # reset the noisy layers
        self.reset_noise()

        return loss.item()

    def update_target_network(self):
        """Update target network parameters"""
        if self.episode_count % self.target_update == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
        self.episode_count += 1

# Initialize the improved agent
agent = NoisyDoubleDQNAgent()

# Modified training function
def train_agent(num_episodes=2000, print_interval=100):
    rewards = []
    losses = []
    epsilons = []

    for episode in range(num_episodes):
        state, _ = env.reset()
        episode_reward = 0
        done = False

        while not done:
            # Select and perform action
            action = agent.select_action(state)
            next_state, reward, done, truncated, _ = env.step(action)

            # Store transition
            agent.store_transition(state, action, reward, next_state, done)

            # Move to next state
            state = next_state
            episode_reward += reward

            # Perform training step
            loss = agent.train_step()
            if loss is not None:
                losses.append(loss)

        # Update target network
        agent.update_target_network()

        # Update exploration rate
        agent.update_epsilon()

        # Store metrics
        rewards.append(episode_reward)
        epsilons.append(agent.epsilon)

        # Print progress
        if (episode + 1) % print_interval == 0:
            avg_reward = np.mean(rewards[-print_interval:])
            avg_loss = np.mean(losses[-print_interval:]) if losses else 0
            print(f"Episode {episode + 1}")
            print(f"Average Reward: {avg_reward:.3f}")
            print(f"Average Loss: {avg_loss:.3f}")
            print(f"Epsilon: {agent.epsilon:.3f}")
            print("-" * 50)

    return rewards, losses, epsilons

## Loading the Trained Q-Policy Model Weights

The policy net state dictionary is the most important aspect of the training of the model. The target network is less useful as it learns without consideration of the states themselves, and is mostly used during training - whereas the policy net is  effective at inference.

In [ ]:
# downloading the .pth file from the google drive link using gdown and the id
filename='Blackjack_Noisy_DoubledQN.pth'
model_weights_link = 'https://drive.google.com/file/d/17sW4uL7M4MbUPSGnrSYjWiJC-1N1l4qd/view?usp=drive_link'
model_weights_id = model_weights_link.split('/')[5]
gdown.download(output=filename,id = model_weights_id, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=17sW4uL7M4MbUPSGnrSYjWiJC-1N1l4qd
To: /content/Blackjack_Noisy_DoubledQN.pth
100%|██████████| 455k/455k [00:00<00:00, 34.4MB/s]


'Blackjack_Noisy_DoubledQN.pth'

In [ ]:
# Initialize the improved agent then loading the state dict from the .pth file
NoisyDDQN = NoisyDoubleDQNAgent()
NoisyDDQN.policy_net.load_state_dict(torch.load(f'/content/{filename}')['policy_net_state_dict'])

<ipython-input-19-da1497d05a62>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  NoisyDDQN.policy_net.load_state_dict(torch.load(f'/content/{filename}')['policy_net_state_di

<All keys matched successfully>

In [ ]:
NoisyDDQN.to_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Evaluation

### Evaluation Metrics of the Trained Model on a New Environment
We create a new test environment using the RGB array with Gymnasium to visualize the agent's environment.

In [ ]:
test_env = gym.make("Blackjack-v1", render_mode='rgb_array')

Our evaluation function removes some components that were used in the training function and mostly captures the rewards - such as whether the agent was able to play optimally in terms of the number of wins, draws, and losses.

In [ ]:
# Evaluate the trained agent
def evaluate_agent_detailed(test_env, trained_agent, n_episodes=200):
    wins = 0
    draws = 0
    losses = 0
    total_rewards = []
    player_sums = []
    dealer_sums = []

    for episode in range(n_episodes):
        state, _ = test_env.reset()
        done = False
        episode_reward = 0

        while not done:
            # Use greedy policy (no exploration)
            with torch.no_grad():
                # Move state_tensor to the same device as the policy_net
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(trained_agent.device)
                action = trained_agent.policy_net(state_tensor).argmax().item()

            state, reward, done, truncated, _ = test_env.step(action)
            episode_reward += reward

        total_rewards.append(episode_reward)
        if reward > 0:
            wins += 1
        elif reward == 0:
            draws += 1
        else:
            losses += 1

        player_sums.append(state[0])  # Final player sum

    print("\nDetailed Evaluation Results:")
    print(f"Number of Episodes: {n_episodes}")
    print(f"Win Rate: {wins/n_episodes*100:.1f}%  ({wins}/{n_episodes})")
    print(f"Draw Rate: {draws/n_episodes*100:.1f}%  ({draws}/{n_episodes})")
    print(f"Loss Rate: {losses/n_episodes*100:.1f}%  ({losses}/{n_episodes})")
    print(f"Average Reward: {np.mean(total_rewards):.3f}")
    print(f"Average Final Player Sum: {np.mean(player_sums):.1f}")

    return total_rewards, player_sums

# Evaluate the trained agent
print("\nEvaluating trained agent...")
eval_rewards, player_sums = evaluate_agent_detailed(test_env, NoisyDDQN, n_episodes=200)


Evaluating trained agent...

Detailed Evaluation Results:
Number of Episodes: 200
Win Rate: 44.0%  (88/200)
Draw Rate: 8.0%  (16/200)
Loss Rate: 48.0%  (96/200)
Average Reward: -0.040
Average Final Player Sum: 19.7


### Visualizing the Agent's Learning with Gymnasium's RGB Environment
This visualization is less useful for evaluation and is mostly useful for interpretting the gameplay. The evaluation output is informative and tells us the statistics of how the agent fares within the evaluation environment. We found that our agent usually performs within the 40-50% win range. This is quite normal as blackjack is at most a game of luck, and in general even the optimal stategy has a .05% disadvantage compared to the dealer/the antagonist of the game.

In [ ]:
# Uses the Gym Monitor wrapper to evalaute the agent and record video
# only one video will be saved
# video of the final episode with the episode trigger
test_env = gym.wrappers.RecordVideo(
    test_env, "./gym_monitor_output", episode_trigger=lambda x: x == 0)

evaluate_agent_detailed(test_env, NoisyDDQN)

test_env.close()

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/gym_monitor_output folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(



Detailed Evaluation Results:
Number of Episodes: 200
Win Rate: 35.0%  (70/200)
Draw Rate: 16.0%  (32/200)
Loss Rate: 49.0%  (98/200)
Average Reward: -0.140
Average Final Player Sum: 19.9


In [ ]:
# play a video using a path to the video
from IPython.display import Video
from base64 import b64encode

def show_video(video_path):
    video_file = Video(video_path, embed=True)
    display(video_file)


In [ ]:
# visualizing the rl-video-episode-0.mp4 in the gym_monitor_output
show_video("./gym_monitor_output/rl-video-episode-0.mp4")

## Strategy Analysis and Interactive Human Feedback Loop with A.I Recommendation
Finally, we implemented a stategy analysis to understand the kinds of decisions that were learned by the agent within the Q-Learning policy matrix. We also implemented a human feedback loop with the agent's action recommendations. The training loop retrains the model after each game and helps to train the model using the human's actions.

### Strategy Analysis

In [ ]:
def analyze_strategy(agent):
    # Common situations in blackjack
    test_states = [
        (16, 10, 0),  # Hard 16 vs dealer 10
        (12, 6, 0),   # Hard 12 vs dealer 6
        (18, 9, 0),   # Hard 18 vs dealer 9
        (11, 10, 0),  # Hard 11 vs dealer 10
        (15, 7, 0),   # Hard 15 vs dealer 7
    ]

    print("\nStrategy Analysis:")
    print("Player Sum | Dealer Card | Action")
    print("-" * 35)

    for player_sum, dealer_card, usable_ace in test_states:
        state = np.array([player_sum, dealer_card, usable_ace])
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(agent.device)
            action = agent.policy_net(state_tensor).argmax().item()
            action_name = "Hit" if action == 1 else "Stand"
            print(f"{player_sum:^10} | {dealer_card:^11} | {action_name:^6}")

# Run strategy analysis
analyze_strategy(NoisyDDQN)


Strategy Analysis:
Player Sum | Dealer Card | Action
-----------------------------------
    16     |     10      |  Hit  
    12     |      6      |  Hit  
    18     |      9      | Stand 
    11     |     10      |  Hit  
    15     |      7      |  Hit  


### Interactive Human Feedback Loop

In [ ]:
def card_name(card):
    """Convert card number to readable format"""
    if card == 1:
        return 'A'
    elif card == 11:
        return 'J'
    elif card == 12:
        return 'Q'
    elif card == 13:
        return 'K'
    else:
        return str(card)

def print_cards(cards, hidden=False):
    """Display cards in readable format"""
    if hidden:
        return f"[{card_name(cards[0])}, ?]"
    return f"[{', '.join(card_name(c) for c in cards)}]"

def get_card_value(card):
    if card == 1:  # Ace
        return 11
    return min(card, 10)

def calculate_hand_value(cards):
    value = sum(get_card_value(card) for card in cards)
    num_aces = cards.count(1)

    # Adjust for aces
    while value > 21 and num_aces:
        value -= 10
        num_aces -= 1

    return value

def play_interactive_blackjack_with_learning(agent):
    print("\nWelcome to Interactive Learning Blackjack!")
    print("The AI will learn from your games.")

    game_memory = []  # Store game experiences
    stats = {'games': 0, 'wins': 0, 'losses': 0, 'draws': 0}

    while True:
        player_cards = []
        dealer_cards = []
        deck = list(range(1, 14)) * 4
        random.shuffle(deck)

        # Initial deal
        player_cards.extend([deck.pop(), deck.pop()])
        dealer_cards.extend([deck.pop(), deck.pop()])

        game_states = []  # Store states for this game

        while True:
            print("\nDealer shows:", print_cards(dealer_cards, hidden=True))
            print("Your cards:", print_cards(player_cards))
            player_value = calculate_hand_value(player_cards)
            print(f"Your total: {player_value}")

            if player_value > 21:
                print("Bust! You lose.")
                stats['losses'] += 1
                break

            # Current state
            current_state = np.array([player_value, get_card_value(dealer_cards[0]), 1 in player_cards])

            # Get AI recommendation
            with torch.no_grad():
                state_tensor = torch.FloatTensor(current_state).unsqueeze(0).to(agent.device)
                q_values = agent.policy_net(state_tensor)
                ai_action = q_values.argmax().item()
                confidence = torch.softmax(q_values, dim=1)[0]
                ai_recommendation = "Hit" if ai_action == 1 else "Stand"
                print(f"\nAI Recommends: {ai_recommendation} (Confidence: {confidence[ai_action]:.2f})")

            action = input("\nYour action (H/S): ").upper()
            while action not in ['H', 'S']:
                action = input("Invalid input. Please enter H or S: ").upper()

            # Store state and action
            game_states.append((
                current_state,
                1 if action == 'H' else 0,
                player_value
            ))

            if action == 'H':
                player_cards.append(deck.pop())
            else:
                break

        # Game ended, calculate final reward
        final_player_value = calculate_hand_value(player_cards)
        dealer_value = play_dealer_hand(dealer_cards, deck)

        if final_player_value <= 21:
            print("\nFinal hands:")
            print(f"Dealer: {print_cards(dealer_cards)} (Total: {dealer_value})")
            print(f"Player: {print_cards(player_cards)} (Total: {final_player_value})")

            if dealer_value > 21:
                print("Dealer busts! You win!")
                reward = 1.0
                stats['wins'] += 1
            elif dealer_value > final_player_value:
                print("Dealer wins!")
                reward = -1.0
                stats['losses'] += 1
            elif dealer_value < final_player_value:
                print("You win!")
                reward = 1.0
                stats['wins'] += 1
            else:
                print("Push (tie)!")
                reward = 0.0
                stats['draws'] += 1
        else:
            reward = -1.0

        # Store experiences for learning
        for state, action, value in game_states:
            agent.memory.push(state, action, reward, state, True)
            # Train on a batch
            if len(agent.memory) >= agent.batch_size:
                loss = agent.train_step()
                if loss:
                    print(f"Training loss: {loss:.4f}")

        stats['games'] += 1
        print("\nCurrent Stats:")
        print(f"Games Played: {stats['games']}")
        print(f"Win Rate: {stats['wins']/stats['games']*100:.1f}%")

        play_again = input("\nPlay again? (Y/N): ").upper()
        if play_again != 'Y':
            break

    print("\nFinal Stats:")
    print(f"Games Played: {stats['games']}")
    print(f"Wins: {stats['wins']}")
    print(f"Losses: {stats['losses']}")
    print(f"Draws: {stats['draws']}")
    print(f"Win Rate: {stats['wins']/stats['games']*100:.1f}%")

    # Save the improved model
    torch.save({
        'policy_net_state_dict': agent.policy_net.state_dict(),
        'optimizer_state_dict': agent.optimizer.state_dict(),
        'epsilon': agent.epsilon,
    }, 'blackjack_dqn_improved.pth')
    print("\nImproved model saved!")

# Helper function for dealer's turn
def play_dealer_hand(dealer_cards, deck):
    dealer_value = calculate_hand_value(dealer_cards)
    while dealer_value < 17:
        dealer_cards.append(deck.pop())
        dealer_value = calculate_hand_value(dealer_cards)
        print(f"Dealer hits: {print_cards(dealer_cards)} (Total: {dealer_value})")
    return dealer_value

# Start the learning interactive game
print("\nStarting interactive learning blackjack game...")
play_interactive_blackjack_with_learning(NoisyDDQN)


Starting interactive learning blackjack game...

Welcome to Interactive Learning Blackjack!
The AI will learn from your games.

Dealer shows: [6, ?]
Your cards: [10, 8]
Your total: 18

AI Recommends: Stand (Confidence: 0.76)

Your action (H/S): S
Dealer hits: [6, 5, 3] (Total: 14)
Dealer hits: [6, 5, 3, Q] (Total: 24)

Final hands:
Dealer: [6, 5, 3, Q] (Total: 24)
Player: [10, 8] (Total: 18)
Dealer busts! You win!

Current Stats:
Games Played: 1
Win Rate: 100.0%

Play again? (Y/N): N

Final Stats:
Games Played: 1
Wins: 1
Losses: 0
Draws: 0
Win Rate: 100.0%

Improved model saved!
